# <center> **Binary Search Tree**
<center> “To be without trees would, in the most literal way, to be without our roots.” - Richard Mabey </center>

### **Important Notes:**
*   **Deadline: March 16 11:59PM (midnight)** 
*   Any questions should be sent to [minhducl@uark.edu](https://) (author) and [sangt@uark.edu](https://)
*   Please **start early** and read the instructions as well as examples. 
*   **We recommend running this file in Google Colab**
*   Run the following code cells for test cases and APIs from previous homeworks


In [2]:
# the data will be downloaded to content directory
! gdown 1HqJDlRWr5Ffo2fc765JDFoh59xj3fhOx --quiet

import json 
with open('content/data.json') as json_file:
    data = json.load(json_file)

#**I. Introduction**
In homework 1, we were working with data structures that are made up of *nodes*. Each node contains links that are either *None* or references to other nodes. In this homework, we cover the binary tree data structure, which has the restriction that every node is pointed to by just one other node (called *parent*), and each node has exactly 2 links called *left* and *right*, that point to nodes that are left child and right child respectively. Let's quickly define the API for this type of node. 




In [18]:
class Node:
  def __init__(self, key: int=None, value: str=None, N: int=0):
    self.key   = key        # key
    self.value = value      # associated values
    self.left  = None       # link to left subtree
    self.right = None       # link to right subtree
    self.N = N              # number of nodes in subtree rooted here

Although a link points to a node, if that node is a root of a subtree, we can see link as pointing to a binary tree. The study of binary tree covers many applications but we will mainly work with *binary search tree*, a special case of binary tree whose ordering support efficient search. 

> **Definition:** a *binary search tree* (BST) is a binary tree where each node has a key that satisfies the restriction that the key in any node is larger than the keys in all nodes in that node's left subtree and smaller than keys in all nodes in that node's right subtree.




<center><img src="https://drive.google.com/uc?export=view&id=1BDRbcJjbrimvjbRMjChq0wPoIh5y6Hiz" width="35%" height="30%">
<figcaption>Figure 1: Example of a binary search tree. This tree has 7 nodes. If we read the key of these nodes from left to right, it would be [3,5,9,12,15,20,22], which follows ascending numerical order. The root of this tree has key 20 and it placed at the top of the tree. </figcaption> </center>




It is often the case that the binary tree is huge with thousands to millions nodes. Without ordering, searching in that tree is considerably inefficient. With BST, a maximum of 2 comparisions is carried out at each node, and we will able to tell which direction to go (left or right?) at that node, making searching much more efficient. We will cover the API of BST and discover one exciting way to maintain the height balance of a BST.

# **Binary Search Tree (BST)**

Binary Search Tree is a very well-known data structure and there is abundance of information that could be found online. We provide the API for it below with some methods that are provided to you as an example. You will see we implement them in recursive fashion as this is the easy way to traverse inside the tree. You will be asked to implement a number of operations on this tree, and the test cases are provided for you to test run it before submission. More instructions will be provided for each operation.

**BST API**
```
class BST:
    __init__(keys: List, vals: List)->None    # read a list to construct the
    size()->int                               # number of nodes in the tree
    insert(key: int, vals: str)->None         # insert a pair of key and value
    printTree()->None                         # print the BST
    height(key: int)->int:                    # height of the tree
    getValue(key: int)->str:                  # get the value associated with the given key
    min()->int                                # return the minimum key
    max()->int                                # return the maximum key
    floor(key: int)->int                      # the largest key smaller to the given key
    ceiling(key: int)->int                    # the smallest key smaller to the given key
    rank(key: int)->int                       # how many keys to the left of the given key
    delete(key: int)->None                    # delete the node that has the given key                          
    AVL_balance()->None                       # balance the tree according to AVL sense
```

One could quickly spot that we define key of integer type and value string type. This is to simplify the implementation, and please bear in mind that they do not have to be integer or string. Python implementation of the API is given below. 

In [19]:
from typing import List


# class definition of binary search tree
# noinspection DuplicatedCode
class BST:
    def __init__(self, keys: List, vals: List):
        self.__root = None
        for (key, val) in zip(keys, vals):
            self.insert(key, val)

    # find the number of nodes in the tree
    def size(self) -> int:
        return self.__size(self.__root)

    # find the number of nodes in the subtree whose root is node
    def __size(self, node: Node) -> int:
        if node is None:
            return 0
        else:
            return node.N

    # insert a new pair of key and associated value to the tree
    def insert(self, key: int, val: str) -> None:
        self.__root = self.__insert(self.__root, key, val)

    def __insert(self, node: Node, key: int, val: str) -> Node:
        # if the node does not exist, add new node
        if node is None: return Node(key, val, 1)
        if key < node.key:
            node.left = self.__insert(node.left, key, val)
        elif key > node.key:
            node.right = self.__insert(node.right, key, val)
        else:
            node.val = val
        node.N = self.__size(node.left) + self.__size(node.right) + 1
        return node

    # print the tree horizontally
    def printTree(self) -> None:
        self.__printTree(self.__root)

    # recursively print the nodes from right to left
    def __printTree(self, node: Node, level: int = 0) -> None:
        if node is not None:
            self.__printTree(node.right, level + 1)
            print(' ' * 6 * level + '-> ' + str(node.key))
            self.__printTree(node.left, level + 1)

    # find the height of a node with given key
    def height(self, key: int) -> int:
        def internal_height(node):
            if node is None:
                return 0
            leftAns = internal_height(node.left)
            rightAns = internal_height(node.right)
            return max(leftAns, rightAns) + 1

        def internal_traverse(node):
            if node is None:
                return None
            if node.key == key:
                return node
            else:
                if key < node.key:
                    temp_one = internal_traverse(node.left)
                elif key > node.key:
                    temp_one = internal_traverse(node.right)
                else:
                    temp_one = None
                return temp_one

        node_found = internal_traverse(self.__root)
        if node_found is not None:
            left_height = internal_height(node_found.left)
            right_height = internal_height(node_found.right)
            return max(left_height, right_height)
        else:
            return -1

    # get the value at a node with a given key
    def getValue(self, key: int) -> str:
        return self.__getValue(self.__root, key)

    def __getValue(self, node: Node, key: int) -> str:
        if node is None: return None
        if key < node.key:
            return self.__getValue(node.left, key)
        elif key > node.key:
            return self.__getValue(node.right, key)
        else:
            return node.value

    def min(self) -> int:
        left_side = self.__root
        while left_side.left is not None:
            left_side = left_side.left
        return left_side.key

    # return the maximum key of the tree
    def max(self) -> int:
        right_side = self.__root
        while right_side.right is not None:
            right_side = right_side.right
        return right_side.key

    def floor(self, key: int):
        if self.__root is None:
            return -1

        if self.__root.key == key:
            return self.__root.key
        elif self.__root.key > key:
            side = self.__root.left
        else:
            side = self.__root.right

        while True:
            if side.left is not None and side.right is not None:
                if side.key < key:
                    side = side.right
                elif side.key > key:
                    side = side.left
            elif side.right is not None:
                if side.key < key:
                    if side.right.key > key:
                        return side.key
                    else:
                        side = side.right
            elif side.left is not None:
                if side.key > key:
                    if side.left.key < key:
                        return side.key
                else:
                    side = side.left
            else:
                if key < side.key:
                    if self.__root.key < key:
                        return self.__root.key
                    else:
                        return None
                elif key == side.key:
                    return key
                else:
                    return side.key

    def ceiling(self, key: int):
        if self.__root is None:
            return -1

        if self.__root.key == key:
            return self.__root.key
        elif self.__root.key > key:
            side = self.__root.left
        else:
            side = self.__root.right

        while True:
            if side.left is not None and side.right is not None:
                if side.key < key:
                    side = side.right
                elif side.key > key:
                    if side.left.key < key < side.key:
                        return side.key
                    else:
                        side = side.left
            elif side.right is not None:
                if side.key > key:
                    if side.right.key > key:
                        return side.key
                    else:
                        side = side.right
                else:
                    side = side.right
            elif side.left is not None:
                if side.key > key:
                    if side.left.key < key:
                        return side.key
                else:
                    side = side.left

            else:
                if key > side.key:
                    if self.__root.key < key:
                        return None
                    else:
                        return self.__root.key
                elif key == side.key:
                    return key
                else:
                    return side.key

    def rank(self, key: int) -> int:
        def find_size(node):
            size = 1
            if node.left is not None and node.key < key:
                size += find_size(node.left)
            if node.right is not None and node.key < key:
                size += find_size(node.right)

            if node.key > key or node.key == key:
                size -= 1
            if node.key > key and node.left is not None:
                size += find_size(node.left)
            return size

        def find_internal_rank(node):
            count = 0
            if key == self.__root.key:
                if node is None:
                    return 0
                else:
                    return find_size(self.__root.left)
            if key > self.__root.key:
                count += 1
                if self.__root.left is not None:
                    count += find_size(self.__root.left)
                if self.__root.right is not None:
                    count += find_size(self.__root.right)
                return count
            if key < self.__root.key:
                if self.__root.left is not None:
                    count += find_size(self.__root.left)
                return count

        return find_internal_rank(self.__root)

    def delete(self, key: int) -> None:
        def min_node(node):
            left_side = node
            while node.left is not None:
                left_side = node.left
            return left_side

        def delete_node(node, temp_key):
            if node is None:
                return node
            if node.key > temp_key:
                node.left = delete_node(node.left, temp_key)
            elif node.key < temp_key:
                node.right = delete_node(node.right, temp_key)
            else:
                if node.left is None:
                    temp = node.right
                    return temp

                elif node.right is None:
                    temp = node.left
                    return temp

                temp_node = min_node(node.right)
                node.key = temp_node.key
                node.right = delete_node(node.right, temp_node.key)
            return node

        self.__root = delete_node(self.__root, key)

    def avlBalance(self) -> None:
        def left_rot(node):

            temp_node = node.right
            temp_left = temp_node.left

            temp_node.left = node
            node.right = temp_left

            if node.left is not None:
                height_one = self.height(node.left.key)
            else:
                height_one = 0
            if node.right is not None:
                height_two = self.height(node.right.key)
            else:
                height_two = 0

            node.N = max(height_one, height_two)

            if temp_node.left is not None:
                height_one = self.height(temp_node.left.key)
            else:
                height_one = 0
            if temp_node.right is not None:
                height_two = self.height(temp_node.right.key)
            else:
                height_two = 0

            temp_node.N = max(height_one, height_two)

            return temp_node

        def right_rot(node):

            temp_node = node.left
            right_temp_node = temp_node.right

            temp_node.right = node
            node.left = right_temp_node

            if node.left is not None:
                height_one = self.height(node.left.key)
            else:
                height_one = 0
            if node.right is not None:
                height_two = self.height(node.right.key)
            else:
                height_two = 0

            node.N = max(height_one, height_two)

            if temp_node.left is not None:
                height_one = self.height(temp_node.left.key)
            else:
                height_one = 0
            if temp_node.right is not None:
                height_two = self.height(temp_node.right.key)
            else:
                height_two = 0

            temp_node.N = max(height_one, height_two)

            return temp_node

        def get_balance(node):
            if self.__root.key != node.key:
                return 0
            return self.height(node.left.key) - self.height(node.right.key)

        def is_balanced(node):
            balance_num = get_balance(node)
            need_left = -1
            need_right = 1

            if balance_num < need_left and node.right.key > node.key:
                node.right = right_rot(node.right)
                return left_rot(node)

            if balance_num < need_left and node.right.key < node.key:
                return left_rot(node)

            if balance_num > need_right and node.left.key < node.key:
                node.left = left_rot(node.left)
                return right_rot(node)

            if balance_num > need_right and node.left.key > node.key:
                return right_rot(node)

            return node

        self.__root = is_balanced(self.__root)


We only provide the methods *size, insert, printTree, getValue*, which are the very basic operations but necessary enough to get us started with the tree. Let's build our first tree in the example.

In [20]:
# build the first tree in the example
bst = BST(**data['example'])

# print the size of the tree
print(f"Size of this tree: {bst.size()} \n")

# print out the tree
print("Visualization of the tree")
bst.printTree()

Size of this tree: 7 

Visualization of the tree
      -> 22
-> 20
                  -> 15
            -> 12
      -> 9
            -> 5
                  -> 3


Let's add a new key 21 to the tree and see how the tree changes.

In [21]:
# add key 21 with associated value "Hello"
bst.insert(21, "Hello")

# print the size of the tree
print(f"Size of this tree: {bst.size()} \n")

# print out the tree
print("Visualization of the tree")
bst.printTree()

Size of this tree: 8 

Visualization of the tree
      -> 22
            -> 21
-> 20
                  -> 15
            -> 12
      -> 9
            -> 5
                  -> 3


What is the value associated with key 15?

In [22]:
# get the value of key 15
print(f"Key 15 has value {bst.getValue(15)}")

Key 15 has value Sang


Now we will need you to help us implement the rest of the API. Let's first talk about finding minimum and maximum key in the tree.

## **Minimum vs Maximum (15 pts)**

<center><img src="https://drive.google.com/uc?export=view&id=1BDRbcJjbrimvjbRMjChq0wPoIh5y6Hiz" width="25%" height="20%">

It can be seen from the example that the minimum key is located all the way to the left. The only attribute in our API is the root node (*self.__root*) and it is set to be private. This means if we want to find the mimimum, we have to traverse from the root to left node and keep traversing left untill the left link is None. Finding the maximum key is similar, moving to the right instead of to the left 

**Test cases for Mimimum**

In [25]:
# test case 0
tc_0 = BST(**data['testcase_0'])
assert(tc_0.min()==1)

# test case 1
tc_1 = BST(**data['testcase_1'])
assert(tc_1.min()==1)

# test case 2
tc_2 = BST(**data['testcase_2'])
assert(tc_2.min()==18)

# test case 3
tc_3 = BST(**data['testcase_3'])
assert(tc_3.min()==2)

# test case 4
tc_4 = BST(**data['testcase_4'])
assert(tc_4.min()==10)

# test case 5
tc_5 = BST(**data['testcase_5'])
assert(tc_5.min()==100)

**Test cases for Maximum**

In [26]:
# test case 0
tc_0 = BST(**data['testcase_0'])
assert(tc_0.max()==15)

# test case 1
tc_1 = BST(**data['testcase_1'])
assert(tc_1.max()==13)

# test case 2
tc_2 = BST(**data['testcase_2'])
assert(tc_2.max()==60)

# test case 3
tc_3 = BST(**data['testcase_3'])
assert(tc_3.max()==20)

# test case 4
tc_4 = BST(**data['testcase_4'])
assert(tc_4.max()==50)

# test case 5
tc_5 = BST(**data['testcase_5'])
assert(tc_5.max()==1100)

## **Floor vs Ceiling (15 pts)**

<center><img src="https://drive.google.com/uc?export=view&id=1BDRbcJjbrimvjbRMjChq0wPoIh5y6Hiz" width="25%" height="20%">

Given a key *k*, floor is the maximum key in the tree that is smaller than k. For example, floor(9) in the example is 5. Ceiling is the opposite of floor, which is the minimum key bigger than k. 

To find floor with a given key *k*, we have the following observations:


1.   if *k* is less than the key at the root of a BST, the floor **must** be in the left subtree. We traverse left.
2.   If *k* is equal to the key at the root, we return *k*.
3.   If **k** is greater than the key at the root of a BST, the floor **could** be in the right subtree. If we cannot find floor in the right subtree, then floor is the key at the root.

The last observation can be verified by finding floor(10). We start at key 20 and traverse left as 10 < 20. At root 9, as 10 > 9 so we traverse right and then traverse left at root 12. However, root 12 has null left link, which means floor(10) is 9. 


**Test cases for floor**

In [27]:
# test case 1
tc_1 = BST(**data['testcase_1'])
assert(tc_1.floor(9)==8)
assert(tc_1.floor(5)==4)
assert(tc_1.floor(10)==10)
assert(tc_1.floor(5)==4)

# test case 4
tc_4 = BST(**data['testcase_4'])
assert(tc_4.floor(55)==50)
assert(tc_4.floor(1)==None)
assert(tc_4.floor(21)==20)
assert(tc_4.floor(28)==25)
assert(tc_4.floor(42)==40)

**Test cases for ceiling**

In [28]:
# test case 1
tc_1 = BST(**data['testcase_1'])
assert(tc_1.ceiling(9)==10)
assert(tc_1.ceiling(4)==4)
assert(tc_1.ceiling(5)==6)
assert(tc_1.ceiling(2)==3)

# test case 4
tc_4 = BST(**data['testcase_4'])
assert(tc_4.ceiling(31)==35)
assert(tc_4.ceiling(41)==45)
assert(tc_4.ceiling(25)==25)
assert(tc_4.ceiling(51)==None)

## **Rank (15 pts)**


<center><img src="https://drive.google.com/uc?export=view&id=1BDRbcJjbrimvjbRMjChq0wPoIh5y6Hiz" width="25%" height="20%">

Given a key *k*, the rank is defined to be the number of keys that is smaller than *k* in the tree. For example, rank(9) = 2 and rank(15) = 4.

To find the rank(k), we have the following observations:


1.   If k is equal to the key at root, we return the number of keys t in the left subtree by computing size(root.left)
2.   If k is less than the key at the root, we return the rank of the key in the left subtree (in recursive fashion of course)
3.   If k is bigger than the ket at the root, we return size(root.left) + 1 + rank of the key is right subtree. 

**Example for observation 1:** rank(20) can be found by calling size(root.left) = 5 \\

**Example for observation 2:** To find rank(9), we start at root 20, and as 9 < 20 so we traverse left to reach root 9, where we call size(root.left) = 2. 

**Example for observation 3:** To find rank(22), we start at root 20, and as 22 > 20 so we return the number of keys in the left subtree of 20 + 1 (root 20 itself) + the number any keys that could be less than 22 in the right subtree of 22. 



In [29]:
# test case 0
tc_0 = BST(**data['testcase_0'])
assert(tc_0.rank(6)==3)
assert(tc_0.rank(11)==5)

# test case 1
tc_1 = BST(**data['testcase_1'])
assert(tc_1.rank(9)==6)
assert(tc_1.rank(4)==2)

# test case 2
tc_2 = BST(**data['testcase_2'])
assert(tc_2.rank(50)==4)
assert(tc_2.rank(22)==1)

# test case 4
tc_4 = BST(**data['testcase_4'])
assert(tc_4.rank(42)==7)
assert(tc_4.rank(21)==3)

## **Height (15 pts)** 
"I want a man that is at least 6 feet!" - most girls in UARK \\

We don't need you to be 6 feet, but we need you to compute the height of the tree given the key *k*. 

<center><img src="https://drive.google.com/uc?export=view&id=1BDRbcJjbrimvjbRMjChq0wPoIh5y6Hiz" width="25%" height="20%">

The height of the tree in the example is 4. The height of the subtree, whose root is 9, is 2. How can we compute the height? We recommend doing it in recursive fashion


1.   If the root is None, return -1
2.   If the root is not None, return 1 + max[height(root.left),height(root.right)]




In [30]:
# test case 0
tc_0 = BST(**data['testcase_0'])
assert(tc_0.height(15)==0)
assert(tc_0.height(7)==4)

# test case 1
tc_1 = BST(**data['testcase_1'])
assert(tc_1.height(6)==1)
assert(tc_1.height(11)==1)

# test case 4
tc_4 = BST(**data['testcase_4'])
assert(tc_4.height(15)==2)
assert(tc_4.height(45)==2)

## **Deletion (20 pts)**

The most difficult BST operation to implement is the *delete* method that removes a key-value pair. We recommend reading the lecture slides that cover the deletion in the BST.

In [31]:
# test case 1
tc_1 = BST(**data['testcase_1'])

print(f"The tree before deleting 6")
print(f"The size of tree before deleting 6 is {tc_1.size()}")
tc_1.printTree()

print(f"\nThe tree after deleting 6")
tc_1.delete(6)
print(f"The size of tree after deleting 6 is {tc_1.size()}")
tc_1.printTree()


The tree before deleting 6
The size of tree before deleting 6 is 9
            -> 13
      -> 11
            -> 10
-> 8
                  -> 7
            -> 6
                  -> 4
      -> 3
            -> 1

The tree after deleting 6
The size of tree after deleting 6 is 9
            -> 13
      -> 11
            -> 10
-> 8
            -> 7
                  -> 4
      -> 3
            -> 1


In [32]:
# test case 1
tc_4 = BST(**data['testcase_4'])

print(f"The tree before deleting 45 and 15")
print(f"The size of tree before deleting 45 and 15 is {tc_4.size()}")
tc_4.printTree()

print(f"\nThe tree after deleting 45 and 15")
tc_4.delete(45)
tc_4.delete(15)
print(f"The size of tree after deleting 45 and 15 is {tc_4.size()}")
tc_4.printTree()

The tree before deleting 45 and 15
The size of tree before deleting 45 and 15 is 9
            -> 50
      -> 45
                  -> 40
            -> 35
-> 30
                  -> 25
            -> 20
      -> 15
            -> 10

The tree after deleting 45 and 15
The size of tree after deleting 45 and 15 is 9
      -> 50
                  -> 40
            -> 35
-> 30
            -> 25
      -> 20
            -> 10


## **Height-Balanced Binary Search Tree (20 pts)**

Write a method *avlBalance()* to balance the tree


In [33]:
# test case 1
tc_1 = BST(**data['testcase_1'])
print("The original tree")
tc_1.printTree()

tc_1.insert(5,"Hello")
print("\nThe tree after adding key 5")
tc_1.printTree()

print("\nThe tree after balancing")
tc_1.avlBalance()
tc_1.printTree()

The original tree
            -> 13
      -> 11
            -> 10
-> 8
                  -> 7
            -> 6
                  -> 4
      -> 3
            -> 1

The tree after adding key 5
            -> 13
      -> 11
            -> 10
-> 8
                  -> 7
            -> 6
                        -> 5
                  -> 4
      -> 3
            -> 1

The tree after balancing
                  -> 13
            -> 11
                  -> 10
      -> 8
            -> 7
-> 6
                  -> 5
            -> 4
      -> 3
            -> 1


In [34]:
# test case 4
tc_4 = BST(**data['testcase_4'])
print("The original tree")
tc_4.printTree()

tc_4.insert(26,"Hello")
tc_4.delete(40)
print("\nThe tree after adding key 26 and deleting key 40")
tc_4.printTree()

print("\nThe tree after balancing")
tc_4.avlBalance()
tc_4.printTree()

The original tree
            -> 50
      -> 45
                  -> 40
            -> 35
-> 30
                  -> 25
            -> 20
      -> 15
            -> 10

The tree after adding key 26 and deleting key 40
            -> 50
      -> 45
            -> 35
-> 30
                        -> 26
                  -> 25
            -> 20
      -> 15
            -> 10

The tree after balancing
                  -> 50
            -> 45
                  -> 35
      -> 30
                  -> 26
            -> 25
-> 20
      -> 15
            -> 10
